In [21]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from clean_data import clean_data, prepare_data
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize

In [2]:
# Read files
train = pd.read_csv('../cars-competition/data/cars_train.csv', index_col='Id')
submission = pd.read_csv('../cars-competition/data/cars_test.csv', index_col='Id')

In [3]:
# Clean Data
train = clean_data(train)
submission = clean_data(submission)

Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!
Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!


In [225]:
# Prepare Data
# columns = ['year','manufacturer','condition','cylinders','fuel','odometer','title_status','transmission','drive','size']
columns = ['year','manufacturer','condition','cylinders','odometer','title_status','transmission','size','lat','long']
X, y = prepare_data(train,columns,typ='train')
X_sub, _ = prepare_data(submission,columns,typ='test')

In [226]:
# Normalize columns
norm = ['year','condition','cylinders','odometer','lat','long']
X[norm] = normalize(X[norm])
X_sub[norm] = normalize(X_sub[norm])

In [227]:
# Feature reduction PCA
n_components = [5,10,15,20,25,30,35,40]
#n_components = [5,10,15]
for n in n_components:
    exec('pca{0} = PCA(n_components = {0})'.format(n))
    exec('X_new{0} = pca{0}.fit_transform(X)'.format(n))

In [228]:
# Split Train and Test

for n in n_components:
    exec('X_train{0}, X_test{0}, y_train{0}, y_test{0} = train_test_split(X_new{0},y,test_size=0.2,random_state=200)'.format(n))

In [229]:
# Linear Regression
for n in n_components:
    exec('lin_reg{0} = LinearRegression()'.format(n))
    exec('lin_reg{0}.fit(X_train{0},y_train{0})'.format(n))

In [230]:
# Predict
for n in n_components:
    exec('y_pred{0} = lin_reg{0}.predict(X_test{0})'.format(n))

In [231]:
# Check error
error = {}
for n in n_components:
    exec('error[{0}] = mean_squared_error(y_test{0}, y_pred{0})'.format(n))

In [232]:
# Check for optimal feature number for PCA: 25
error

{5: 69546349771483.164,
 10: 69546149713189.8,
 15: 69546330484429.34,
 20: 69544599494394.445,
 25: 69545433686494.29,
 30: 69545392093740.62,
 35: 69546140649449.71,
 40: 69546279029515.26}

In [ ]:
# 32605224719649.1 30-bsf-2
# 69543350910505.22

In [152]:
# Try optimal Linear Regression for submission
# PCA
X_sub30 = pca30.transform(X_sub)
# Predict
y_sub30 = lin_reg30.predict(X_sub30)
y_sub30

array([ 71625.29995414, 185576.2054675 ,  16293.36125663, ...,
       118887.90367737,  59090.0964403 , 157017.83206255])

In [153]:
# Getting absolute values
y_sub30 = abs(y_sub30)
y_sub30

array([ 71625.29995414, 185576.2054675 ,  16293.36125663, ...,
       118887.90367737,  59090.0964403 , 157017.83206255])

In [154]:
# Prepare for submission
sub30 = submission
sub30['price'] = y_sub30
sub30 = sub30['price'] 
sub30.to_csv('../cars-competition/data/sub30_ln_bsf_2.csv', header=True, index=True)